<a href="https://colab.research.google.com/github/110805/Tense_Conversion/blob/master/CVAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/110805/Tense_Conversion.git
%cd Tense_Conversion/

Cloning into 'Tense_Conversion'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 26 (delta 10), reused 10 (delta 2), pack-reused 0
Unpacking objects: 100% (26/26), done.
/content/Tense_Conversion


In [14]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import time
import math
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
from os import system
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu

"""========================================================================================
The sample.py includes the following template functions:

1. Encoder, decoder
2. Training function
3. BLEU-4 score function
4. Gaussian score function

You have to modify them to complete the lab.
In addition, there are still other functions that you have to 
implement by yourself.

1. The reparameterization trick
2. Your own dataloader (design in your own way, not necessary Pytorch Dataloader)
3. Output your results (BLEU-4 score, conversion words, Gaussian score, generation words)
4. Plot loss/score
5. Load/save weights

There are some useful tips listed in the lab assignment.
You should check them before starting your lab.
========================================================================================"""

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SOS_token = 0
EOS_token = 1
#----------Hyper Parameters----------#
hidden_size = 256
latent_size = 32
#The number of vocabulary
vocab_size = 28
tense_size = 4
conditional_size = 8
teacher_forcing_ratio = 1.0
#empty_input_ratio = 0.1
LR = 0.01
MAX_LENGTH = 20



################################
#Example inputs of compute_bleu
################################
#The target word
reference = 'accessed'
#The word generated by your model
output = 'access'

#compute BLEU-4 score
def compute_bleu(output, reference):
    cc = SmoothingFunction()
    if len(reference) == 3:
        weights = (0.33,0.33,0.33)
    else:
        weights = (0.25,0.25,0.25,0.25)
    return sentence_bleu([reference], output,weights=weights,smoothing_function=cc.method1)

"""============================================================================
example input of Gaussian_score

words = [['consult', 'consults', 'consulting', 'consulted'],
['plead', 'pleads', 'pleading', 'pleaded'],
['explain', 'explains', 'explaining', 'explained'],
['amuse', 'amuses', 'amusing', 'amused'], ....]

the order should be : simple present, third person, present progressive, past
============================================================================"""

def Gaussian_score(words):
    words_list = []
    score = 0
    yourpath = '/content/Tense_Conversion/train.txt' #should be your directory of train.txt
    with open(yourpath,'r') as fp:
        for line in fp:
            word = line.split(' ')
            word[3] = word[3].strip('\n')
            words_list.extend([word])
        for t in words:
            for i in words_list:
                if t == i:
                    score += 1
    return score/len(words)

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def get_train_pair(i, Data):
    input_tensor = []
    target_tensor = []
    
    for char in Data[i]:
        input_tensor.append(ord(char)-95)
        target_tensor.append(ord(char)-95)

    target_tensor.append(EOS_token)
    return (torch.tensor(input_tensor, dtype=torch.long).view(-1, 1), torch.tensor(target_tensor, dtype=torch.long).view(-1, 1), i%4)

#Encoder
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.condition_embedding = nn.Embedding(tense_size, conditional_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size + conditional_size)
        self.hidden2mean = nn.Linear(hidden_size + conditional_size, latent_size)
        self.hidden2var = nn.Linear(hidden_size + conditional_size, latent_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(len(input), 1, -1)
        output = embedded
        output, (hidden, cell) = self.lstm(output, hidden)

        mean = self.hidden2mean(hidden)
        logvar = self.hidden2var(hidden)
        latent = self.reparameterization(mean, logvar)

        return latent, self.KL_loss(mean, logvar)

    def reparameterization(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn(1, 1, latent_size, device=device) # sample a gaussain noise from N(0, I)
        z = mean + std*eps
        return z

    def KL_loss(self, mean, logvar):
        return -0.5 * torch.sum(1 + logvar - mean*mean - torch.exp(logvar))

    def initHidden(self, tense):
        c = torch.tensor(tense, device=device)
        condition = self.condition_embedding(c).view(1, 1, -1)
        hidden = torch.cat((torch.zeros(1, 1, self.hidden_size, device=device), condition), 2)
        return hidden

    def initCell(self):
        return torch.zeros(1, 1, self.hidden_size + conditional_size, device=device)

#Decoder
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.condition_embedding = nn.Embedding(tense_size, conditional_size)
        self.latent2hidden = nn.Linear(latent_size, hidden_size)
        self.latent2cell = nn.Linear(latent_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size + conditional_size)
        self.out = nn.Linear(hidden_size + conditional_size, output_size)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = self.out(output[0])
        return output, hidden

    def initHidden(self, latent, tense):
        c = torch.tensor(tense, device=device)
        condition = self.condition_embedding(c).view(1, 1, -1)
        hidden = self.latent2hidden(latent).view(1, 1, -1)
        hidden = torch.cat((hidden, condition), 2)
        return hidden
    
    def initCell(self, latent, tense):
        c = torch.tensor(tense, device=device)
        condition = self.condition_embedding(c).view(1, 1, -1)
        cell = self.latent2cell(latent).view(1, 1, -1)
        cell = torch.cat((cell, condition), 2)
        return cell

def evaluate(encoder, decoder, input_string, tense, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = []
        for input_char in input_string:
            input_tensor.append(ord(input_char)-95)

        input_tensor = torch.tensor(input_tensor, dtype=torch.long).view(-1, 1)
        input_tensor = input_tensor.to(device)

        encoder_hidden = encoder.initHidden(tense[0])
        encoder_cell = encoder.initCell()

        latent, KL_loss = encoder(input_tensor, (encoder_hidden, encoder_cell))

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = decoder.initHidden(latent, tense[1])
        decoder_cell = decoder.initCell(latent, tense[1])
        decoder_hidden = (decoder_hidden, decoder_cell)
        decoded_words = []
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)

            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                break
            else:
                decoded_words.append(chr(topi.item()+95))

            decoder_input = topi.squeeze().detach()

        pred = ''
        for i in range(len(decoded_words)):
            pred += decoded_words[i]

        return pred

def evalTestdata(encoder, decoder):
    score = 0
    with open('test.txt', 'r') as f:
        all_lines = f.readlines()

    Input = []
    Target = []
    tenses = [[0,3], [0,2], [0,1], [0,1], [3,1], [0,2], [3,0], [2,0], [2,3], [2,1]]
    for line in all_lines:
        if line[-1] == '\n':
            line = line[:-1]

        words = line.split(' ')
        Input.append(words[0])
        Target.append(words[1])
    
    for i in range(len(Input)):
        output = evaluate(encoder, decoder, Input[i], tenses[i])
        print('input: {}'.format(Input[i]))
        print('target: {}'.format(Target[i]))
        print('pred: {}'.format(output))
        
        if len(output) != 0:
            score += compute_bleu(output, Target[i])
        else:
            score += compute_bleu('', Target[i]) # predict empty string
        
        print('--------------------')
    b_score = score/len(Input)
    score = 0
    if b_score>=0.4 and b_score<0.6:
        score = 8
    elif b_score>=0.6 and b_score<0.7:
        score = 9
    elif b_score>=0.7:
        score = 10
    else:
        score = 0

    return b_score, score

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, tense, iters):
    encoder_hidden = encoder.initHidden(tense)
    encoder_cell = encoder.initCell()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    target_length = target_tensor.size(0)

    loss = 0
    #----------sequence to sequence part for encoder----------#
    latent, KL_loss = encoder(input_tensor, (encoder_hidden, encoder_cell))

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = decoder.initHidden(latent, tense)
    decoder_cell = decoder.initCell(latent, tense)
    decoder_hidden = (decoder_hidden, decoder_cell)
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    #----------sequence to sequence part for decoder----------#
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break
    
    beta = iters/50000 # KL cost annealing
    if beta >= 0.8:
        ㄔ
    beta = min(beta, 0.4)

    Loss = loss + beta * KL_loss
    Loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length, KL_loss

def trainIters(encoder, decoder, n_iters, learning_rate=LR):
    start = time.time()
    plot_losses = []
    plot_KL = []
    BLEU_scores = []
    print_loss = 0
    print_KL = 0
    print_every = 5000
    best_score = 0

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    with open('train.txt', 'r') as f:
        all_lines = f.readlines()

    Data = []
    for line in all_lines:
        if line[-1] == '\n':
            line = line[:-1]

        words = line.split(' ')
        for word in words:
            Data.append(word)

    training_pairs = [get_train_pair(random.choice(range(len(Data))), Data) for i in range(n_iters)]
    criterion = nn.CrossEntropyLoss()
    
    for iters in range(1, n_iters):
        training_pair = training_pairs[iters - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        tense = training_pair[2]
        input_tensor = input_tensor.to(device)
        target_tensor = target_tensor.to(device)

        loss, KL_loss = train(input_tensor, target_tensor, encoder,
                    decoder, encoder_optimizer, decoder_optimizer, criterion, tense, iters - 1)
    
        print_loss += loss
        print_KL += KL_loss
        if iters % print_every == 0:
            print_loss_avg = print_loss / print_every
            print_KL_avg = print_KL / print_every
            plot_losses.append(print_loss_avg)
            plot_KL.append(print_KL_avg)
            print_loss = 0
            print_KL = 0

            bleu_score, b = evalTestdata(encoder, decoder)
            gaussian_score, g = generating_words(decoder)
            BLEU_scores.append(bleu_score)
            print('%s (%d %d%%) %.4f %.4f %.4f %.4f' % (timeSince(start, iters / n_iters),
                                    iters, iters / n_iters * 100, print_loss_avg, print_KL_avg, bleu_score, gaussian_score))
            
            if b + g > best_score:
                best_score = b + g
                torch.save(encoder.state_dict(), 'encoder.pkl')
                torch.save(decoder.state_dict(), 'decoder.pkl')
        
    plt.figure(1)
    plt.plot(range(len(plot_losses)), plot_losses)
    plt.xlabel('Iters/5000')
    plt.ylabel('CrossEntropyLoss')
    plt.savefig('CELoss')

    plt.figure(2)
    plt.plot(range(len(plot_KL)), plot_KL)
    plt.xlabel('Iters/5000')
    plt.ylabel('KL_Loss')
    plt.savefig('KLD')

    plt.figure(3)
    plt.plot(range(len(BLEU_scores)), BLEU_scores)
    plt.xlabel('Iters/5000')
    plt.ylabel('BLEU_scores')
    plt.savefig('BLEU_scores')

def generating_words(decoder):
    n = 100
    words = []
    for i in range(n):
        eps = torch.rand(1, 1, latent_size, device=device)
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        word = []
        for t in range(tense_size):
            decoder_hidden = decoder.initHidden(eps, t)
            decoder_cell = decoder.initCell(eps, t)
            decoder_hidden = (decoder_hidden, decoder_cell)
            decoded_words = []
            for di in range(MAX_LENGTH):
                decoder_output, decoder_hidden = decoder(
                    decoder_input, decoder_hidden)

                topv, topi = decoder_output.data.topk(1)
                if topi.item() == EOS_token:
                    break
                else:
                    decoded_words.append(chr(topi.item()+95))

                decoder_input = topi.squeeze().detach()

            pred = ''
            for i in range(len(decoded_words)):
                pred += decoded_words[i]

            word.append(pred)
        
        words.append(word)
    
    print(words)
    score = 0
    g_score = Gaussian_score(words)
    if g_score >= 0.05 and g_score < 0.2:
        score = 16
    elif g_score >= 0.2 and g_score < 0.3:
        score = 18
    elif g_score >= 0.3:
        score = 20
    else:
        score = 0

    return Gaussian_score(words), score

encoder1 = EncoderRNN(vocab_size, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, vocab_size).to(device)
trainIters(encoder1, decoder1, 450000)
decoder = DecoderRNN(hidden_size, vocab_size).to(device)
decoder.load_state_dict(torch.load('decoder.pkl'))
generating_words(decoder)

Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x7f5257984d08>
Traceback (most recent call last):
  File "/usr/lib/python3.6/_weakrefset.py", line 38, in _remove
    def _remove(item, selfref=ref(self)):
KeyboardInterrupt


KeyboardInterrupt: ignored